In [ ]:
# -*- coding: utf-8 -*-
# =============================================================================
# PRUEBA DEL MODELO PROPHET A NIVEL DE PAÍS
# =============================================================================
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from prophet import Prophet
import warnings

warnings.filterwarnings('ignore')

# ASUMIMOS QUE 'df_featured' y 'df_prophet_base' YA EXISTEN DEL SCRIPT ANTERIOR

# Funciones de métricas y optimización de Prophet
def mape(y_true, y_pred, eps=1e-8):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    mask = y_true != 0
    if not np.any(mask): return 0.0
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100

def smape(y_true, y_pred, eps=1e-8):
    return np.mean(2.0 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred) + eps)) * 100

def compute_metrics(y_true, y_pred):
    y_true_orig = np.expm1(y_true)
    y_pred_orig = np.expm1(y_pred)
    return {
        "MAE": mean_absolute_error(y_true_orig, y_pred_orig),
        "RMSE": np.sqrt(mean_squared_error(y_true_orig, y_pred_orig)),
        "R2": r2_score(y_true_orig, y_pred_orig),
        "MAPE": mape(y_true_orig, y_pred_orig),
        "SMAPE": smape(y_true_orig, y_pred_orig)
    }

def optimize_prophet_country(train_df, regressors):
    # Usamos una configuración estándar para Prophet, ya que la optimización es costosa por país.
    # Se puede expandir con un grid de hiperparámetros si es necesario.
    m = Prophet(yearly_seasonality=True, weekly_seasonality=False, daily_seasonality=False)
    if regressors:
        for r in regressors:
            m.add_regressor(r)
    m.fit(train_df)
    return m

# =============================================================================
# PIPELINE DE PROPHET POR PAÍS CON VALIDACIÓN CRUZADA
# =============================================================================
print("🚀 Iniciando pipeline de Prophet a nivel de país...")

n_splits = 5 # Usaremos 5 pliegues, pero ajustado a series más cortas
unique_countries = df_featured['country'].unique()
all_country_results = []
regressors_prophet = [r for r in df_prophet_base.columns if r not in ['ds', 'y']]


# Bucle exterior: Itera sobre cada país
for country in tqdm(unique_countries, desc="Procesando países"):
    
    df_country = df_prophet_base[df_featured['country'] == country].copy().reset_index(drop=True)
    
    # Se necesita un mínimo de datos para una validación cruzada de 5 pliegues
    if len(df_country) < n_splits + 1:
        print(f"⚠️  Saltando {country}: datos insuficientes ({len(df_country)} registros).")
        continue

    # Bucle interior: Validación cruzada para el país actual
    tscv_country = TimeSeriesSplit(n_splits=n_splits)
    for fold, (train_idx, test_idx) in enumerate(tscv_country.split(df_country)):
        
        # 1. División de datos para el fold actual
        prophet_train = df_country.iloc[train_idx]
        prophet_test = df_country.iloc[test_idx]

        if prophet_train.empty or prophet_test.empty:
            continue

        # 2. Preprocesamiento (Escalado de regresores) DENTRO del fold
        if regressors_prophet:
            scaler = StandardScaler()
            prophet_train[regressors_prophet] = scaler.fit_transform(prophet_train[regressors_prophet])
            prophet_test[regressors_prophet] = scaler.transform(prophet_test[regressors_prophet])
        
        # 3. Entrenamiento del modelo
        try:
            model = optimize_prophet_country(prophet_train, regressors_prophet)
            
            # 4. Predicción
            future_df = model.make_future_dataframe(periods=0) # Usamos solo las fechas de test
            future_df = prophet_test.drop(columns='y')
            forecast = model.predict(future_df)
            
            # 5. Evaluación
            metrics = compute_metrics(prophet_test["y"].values, forecast["yhat"].values)
            metrics['country'] = country
            metrics['fold'] = fold + 1
            all_country_results.append(metrics)
        
        except Exception as e:
            print(f"❌ Error procesando {country} en fold {fold+1}: {e}")


# =============================================================================
# 8. RESUMEN DE RESULTADOS DE PROPHET POR PAÍS
# =============================================================================
if all_country_results:
    df_results_country = pd.DataFrame(all_country_results)

    print("\n\n📊 Resumen de Rendimiento de Prophet por País (media de métricas):")
    summary_country = df_results_country.groupby("country")[["MAE", "RMSE", "R2"]].mean().sort_values("R2", ascending=False)
    print(summary_country)

    print("\n\n📈 Rendimiento Promedio General de Prophet (a través de todos los países):")
    print(summary_country.mean())

    # Visualización de la distribución del R2 entre países
    plt.figure(figsize=(12, 8))
    sns.boxplot(x=df_results_country['R2'])
    plt.title('Distribución del R² de Prophet entre todos los países')
    plt.xlabel('R² Score')
    plt.show()
else:
    print("\nNo se generaron resultados para analizar.")



SyntaxError: unmatched ']' (3113550249.py, line 136)